In [1]:
import os

In [3]:
sorted(os.listdir("rawfiles"))

['Cnaes.csv',
 'Empresas0.csv',
 'Empresas1.csv',
 'Empresas2.csv',
 'Empresas3.csv',
 'Empresas4.csv',
 'Empresas5.csv',
 'Empresas6.csv',
 'Empresas7.csv',
 'Empresas8.csv',
 'Empresas9.csv',
 'Estabelecimentos0.csv',
 'Estabelecimentos1.csv',
 'Estabelecimentos2.csv',
 'Estabelecimentos3.csv',
 'Estabelecimentos4.csv',
 'Estabelecimentos5.csv',
 'Estabelecimentos6.csv',
 'Estabelecimentos7.csv',
 'Estabelecimentos8.csv',
 'Estabelecimentos9.csv',
 'Motivos.csv',
 'Municipios.csv',
 'Naturezas.csv',
 'Paises.csv',
 'Qualificacoes.csv',
 'Simples.csv',
 'Socios0.csv',
 'Socios1.csv',
 'Socios2.csv',
 'Socios3.csv',
 'Socios4.csv',
 'Socios5.csv',
 'Socios6.csv',
 'Socios7.csv',
 'Socios8.csv',
 'Socios9.csv']